# Training LSTM-250 for Phoneme Recognition on TIMIT

## Goals

- Loading feature datasets exported by previous notebook
- Model design
- Dataloader design
- Training dashboard
- Training
- Validate trained model and test dataset generation
- Exporting trained model and test dataset as SQLite3 databse for C implementation

# Environment Setup

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import IPython.display as ipd
from tqdm.auto import tqdm


if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('Using PyTorch version:', torch.__version__, ' Device:', device)

Using PyTorch version: 2.0.1  Device: cuda


# Loading Feature Datasets

In [2]:
train_ds_path = './session/train-features.pt'
test_ds_path = './session/test-features.pt'
train_feat_dict = torch.load(train_ds_path)
test_feat_dict  = torch.load(test_ds_path)

In [3]:
print(train_feat_dict.keys())
print(train_feat_dict['note'])
print(train_feat_dict['data-schema'])

Train_feat = train_feat_dict['data']
print('Train_feat:', len(Train_feat))
print('Feature-len:', len(Train_feat[0][1]))

dict_keys(['note', 'data-schema', 'data'])

Notes:
- Feature record: (phone, feature_sequence)
- feature_seqence: list(feature_vector)
- len(feature_vector): 123

Features are extracted using following parameters
F_n_mfcc    : 40
F_fft_window: 256
F_hop_length: 64
F_n_mels    : 128

(phoneme, feature-sequence)
Train_feat: 51848
Feature-len: 123


In [4]:
print(test_feat_dict.keys())
print(test_feat_dict['note'])
print(test_feat_dict['data-schema'])

Test_feat = test_feat_dict['data']
print('Train_feat:', len(Test_feat))
print('Feature-len:', len(Test_feat[0][1]))

dict_keys(['note', 'data-schema', 'data'])

Notes:
- Feature record: (phone, feature_sequence)
- feature_seqence: list(feature_vector)
- len(feature_vector): 123

Features are extracted using following parameters
F_n_mfcc    : 40
F_fft_window: 256
F_hop_length: 64
F_n_mels    : 128

(phoneme, feature-sequence)
Train_feat: 7333
Feature-len: 123


In [5]:
# Delete redundant names to avoid confusions
del train_ds_path, test_ds_path, train_feat_dict, test_feat_dict

## Show Some Data points

**NOTE:**
- Observe that the feature sequence layout is (feature-vector, sequence-point)
- For LSTM training, it needs to be transposed to make it (sequence-point, feature-vector)

In [8]:
# Given an item from Train/Test_feat, shows some information
# feat_item: (phone, feature-sequence)
def showFeatInfo(feat_item, end=''):    
    print('item type:', type(feat_item))
    phone, feat_seq = feat_item
    print('phone:', phone)
    print('feat-seq type:', type(feat_seq))
    print('feat_seq:', feat_seq.shape, end)


# show a few items
showFeatInfo(Train_feat[0], end='\n')
showFeatInfo(Train_feat[3], end='\n')

showFeatInfo(Test_feat[0], end='\n')
showFeatInfo(Test_feat[3], end='\n')

item type: <class 'list'>
phone: h#
feat-seq type: <class 'numpy.ndarray'>
feat_seq: (123, 42) 

item type: <class 'list'>
phone: l
feat-seq type: <class 'numpy.ndarray'>
feat_seq: (123, 17) 

item type: <class 'list'>
phone: h#
feat-seq type: <class 'numpy.ndarray'>
feat_seq: (123, 38) 

item type: <class 'list'>
phone: s
feat-seq type: <class 'numpy.ndarray'>
feat_seq: (123, 17) 



## Transpose Feature Sequences

In [9]:
# Given a feature dataset, transposes the feature sequences
def transposeFeatSeq(feat_ds):
    for item in feat_ds:
        item[1] = item[1].T
        
        
# Transpose the feature datasets
transposeFeatSeq(Train_feat)
transposeFeatSeq(Test_feat)

# show a few item info
showFeatInfo(Train_feat[0], end='\n')
showFeatInfo(Train_feat[3], end='\n')

showFeatInfo(Test_feat[0], end='\n')
showFeatInfo(Test_feat[3], end='\n')

item type: <class 'list'>
phone: h#
feat-seq type: <class 'numpy.ndarray'>
feat_seq: (42, 123) 

item type: <class 'list'>
phone: l
feat-seq type: <class 'numpy.ndarray'>
feat_seq: (17, 123) 

item type: <class 'list'>
phone: h#
feat-seq type: <class 'numpy.ndarray'>
feat_seq: (38, 123) 

item type: <class 'list'>
phone: s
feat-seq type: <class 'numpy.ndarray'>
feat_seq: (17, 123) 



# Model Design

# Dataloader Design

# Setting up Training Dashboard

# Run Training

# Validate Trained Model

# Export Model and Test Dataset